In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 10 22:52:48 2022

@author: Mauricio Alvarado
"""

import pandas as pd
from openbb_terminal.api import openbb


"""""""""
Para más información sobre OpenBB Terminal, ver la página: https://openbb.co/products/terminal
Documentación sobre los algoritmos: https://openbb-finance.github.io/OpenBBTerminal/terminal/forecast/
Cualquier dificultad, comunicarse con soporte. Tienen una comunidad en Discord muy activa!
Aproximadamente, cada serie puede tomar 3 minutos para los 6 algoritmos

"""""""""


# Definiendo los tickers. Más en https://finance.yahoo.com/
variables={
    "CL=F": "Crude Oil Futures",
    "GC=F": "Gold Futures",
    "SL=F": "Silver Futures",
    "HG=F": "Copper Futures",
    "ZS=F": "Soybean Futures",
    "ZC=F": "Corn Futures",
    "^FVX": "Treasury Yield 5y", 
    "^TNX": "Treasury Yield 10y",
    "^TYX": "Treasury Yield 30y",
    "DX-Y.NYB": "US Dollar/USDX - Index",
    "PEN=X": "USD/PEN",
    "JPY=X": "USD/JPY",
    "EUR=X": "USD/EUR"
}
tickers = variables.keys()


# Definiciones de los parámetros
value_req = "Close"      # Defino si deseo el dato de "Open", "Low", "High", "Close"
days_to_predict = 30     # Días hacia adelante en la proyección. Lo máximo son 30 días


"""""""""
Algoritmos para la proyección:
--------------------------------------------------------------------
* expo -> Probabilistic Exponential Smoothing -> Resultado Estocástico
* linrerg -> Probabilistic Linear Regression (Logístico) -> Resultado Estocástico
* regr -> Regression -> Resultado Determinístico
* rnn -> Probabilistic Recurrent Neural Network (RNN, LSTM, GRU) -> Resultado Estocástico
* nbeats -> Neural Bayesian Estimation (feat. Past covariates) -> Resultado Determinístico
* tcn -> Temporal Convolutional Neural Network (feat. Past covariates) -> Resultado Determinístico


Algoritmos no usados, pero que pueden ser implementados:
--------------------------------------------------------------------
* theta -> Theta Method
* brnn -> Block Recurrent Neural Network (RNN, LSTM, GRU) (feat. Past covariates)
* trans -> Transformer model (feat. Past covariates)
* tft -> Temporal Fusion Transformer (feat. Past covariates)
"""""""""

# Defino los DF para cada uno de los algoritmos
expo_df = pd.DataFrame()
linregr_df = pd.DataFrame()
regr_df = pd.DataFrame()
rnn_df = pd.DataFrame()
nbeats_df = pd.DataFrame()
tcn_df = pd.DataFrame()


# Importación y estimación
for i in tickers:
    serie_importada = openbb.stocks.load(i)[[value_req]]
    serie_importada = serie_importada.rename({value_req: i}, axis=1).reset_index()
    serie_importada = serie_importada[serie_importada.index >= '2021-01-01']

    # Estimación
    expo_f = openbb.forecast.expo(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]
    linregr_f = openbb.forecast.linregr(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]
    regr_f = openbb.forecast.regr(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]
    rnn_f = openbb.forecast.rnn(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]
    nbeats_f = openbb.forecast.nbeats(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]
    tcn_f = openbb.forecast.tcn(data=serie_importada, target_column=i, n_predict=days_to_predict)[2]

    # Conversión a DataFrames
    expo_f = expo_f.quantile_df().reset_index().rename({f"{i}_0.5": i}, axis=1)
    linregr_f = linregr_f.quantile_df().reset_index().rename({f"{i}_0.5": i}, axis=1)
    regr_f = regr_f.pd_dataframe().reset_index()
    rnn_f = rnn_f.quantile_df().reset_index().rename({f"{i}_0.5": i}, axis=1)
    nbeats_f = nbeats_f.pd_dataframe().reset_index()
    tcn_f = tcn_f.pd_dataframe().reset_index()

    # Juntando todas las series en un único DataFrame
    expo_df = pd.concat([expo_df, expo_f]) if expo_df.empty is True else pd.merge(expo_df, expo_f[["date", i]], how="left")
    linregr_df = pd.concat([linregr_df, linregr_f]) if linregr_df.empty is True else pd.merge(linregr_df, linregr_f[["date", i]], how="left")
    regr_df = pd.concat([regr_df, regr_f]) if regr_df.empty is True else pd.merge(regr_df, regr_f[["date", i]], how="left")
    rnn_df = pd.concat([rnn_df, rnn_f]) if rnn_df.empty is True else pd.merge(rnn_df, rnn_f[["date", i]], how="left")
    nbeats_df = pd.concat([nbeats_df, nbeats_f]) if nbeats_df.empty is True else pd.merge(nbeats_df, nbeats_f[["date", i]], how="left")   
    tcn_df = pd.concat([tcn_df, tcn_f]) if tcn_df.empty is True else pd.merge(tcn_df, tcn_f[["date", i]], how="left") 


# Renombrando cada una de las columnas para todas los DataFrames
expo_df = expo_df.rename(variables, axis=1)
linregr_df = linregr_df.rename(variables, axis=1)
regr_df = regr_df.rename(variables, axis=1)
rnn_df = rnn_df.rename(variables, axis=1)
nbeats_df = nbeats_df.rename(variables, axis=1)
tcn_df = tcn_df.rename(variables, axis=1)


# Índice para la fecha
expo_df.set_index("date", inplace=True)
linregr_df.set_index("date", inplace=True)
regr_df.set_index("date", inplace=True)
rnn_df.set_index("date", inplace=True)
nbeats_df.set_index("date", inplace=True)
tcn_df.set_index("date", inplace=True)


# Exportación de los resultados. Crear una carpeta llamada "resultados"!
expo_df.to_excel("./resultados/expo_forecast.xlsx")
linregr_df.to_excel("./resultados/linregr_forecast.xlsx")
regr_df.to_excel("./resultados/regr_forecast.xlsx")
rnn_df.to_excel("./resultados/rnn_forecast.xlsx")
nbeats_df.to_excel("./resultados/nbeats_forecast.xlsx")
tcn_df.to_excel("./resultados/tcn_forecast.xlsx")